In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

from pandas_datareader import data
from scipy.stats import norm

%matplotlib inline

In [2]:
def cumm_prod(price_df, start=None, end=None):
    """
    Calculating the cummulative product of the percentage change to normalize the return and be
    able to compare the stocks. 

    """
    if start == None and end == None:
        return_series = (price_df.pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
    else:
        if end == None:
            end = date.today()
        return_series = (price_df.loc[start:end].pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
        
    
    return return_series

def annualized_return(adjClose_return_df, n_years=None):
    """
    n_years needs to represent the period of the adjClose_return_df.
        If it's 1 year worth of cummulative return, you input 1 for the 1y annualized returns.
        If it's 3 years worth of cummulative return, you input 3 for the long period annualized returns.

        (1+total return)pow(1/N) - 1) where N = number of years
    """
    if n_years == None or n_years == 0:
        print("Input How many years does your data covers!")
    else:
        annualized_return = ((1 + adjClose_return_df.tail(1))**(1/n_years)-1) * 100
        return annualized_return

def annualized_historical_volatility(closePrice_df, start=None, end=None, n_days=None):
    """
    start (str): To calculate HV from a start date to today
    end (str): requires start, to calculate HV for a specific period
    n_days (int): To calculate HV for the past N days

    """

    if n_days != None and n_days != 0:
        get_n_data = closePrice_df.tail(n_days)
        hv = np.sqrt(np.log(get_n_data / get_n_data.shift(1)).var()) * np.sqrt(252)
    elif start != None:
        if end == None:
            end = date.today()
            get_n_data = closePrice_df.loc[start:end]
            hv = np.sqrt(np.log(get_n_data / get_n_data.shift(1)).var()) * np.sqrt(252)
        else:
            get_n_data = closePrice_df.loc[start:end]
            hv = np.sqrt(np.log(get_n_data / get_n_data.shift(1)).var()) * np.sqrt(252)       
    else:
        # Get HV for the whole data available
         hv = np.sqrt(np.log(closePrice_df / closePrice_df.shift(1)).var()) * np.sqrt(252)
    
    return hv

def rolling_historical_volatility(closePrice_df, start=None, end=None, window=None):

    get_data = closePrice_df.loc[start:end]
    cchv = np.sqrt(252) * pd.DataFrame.rolling(np.log(get_data / get_data.shift(1)),window=window).std()

    return cchv

In [8]:
close_panel = pd.read_csv('data_collection_cleaning/prices_collection_cleaning/Cleaned_closeP_5y.csv', index_col='Date')
AdjClose_panel = pd.read_csv('data_collection_cleaning/prices_collection_cleaning/Cleaned_AdjCloseP_5y.csv', index_col='Date')
stock_full_Data = pd.read_csv('Final_stock_data.csv')

In [13]:
stock_full_Data.head()

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,marketcap_cat,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.150,0.103398,1.308405e+10,Large,1.25,9.550,0.1525,13.932986,22.424082,109.188247,12.97,0.171149
1,ABT,Abbott Laboratories,Healthcare,Increase,Above Fair Price,OverValued,Average,118.290,0.181179,2.091710e+11,Large,0.74,3.940,0.1318,22.924420,30.022842,51.234756,25.96,0.109154
2,ACN,Accenture plc,Technology,Increase,Above Fair Price,OverValued,Sustainable,323.905,0.208836,2.047083e+11,Large,1.21,9.613,0.1135,31.025385,33.694477,121.148794,9.45,0.166287
3,ADM,Archer-Daniels-Midland Company,Consumer Defensive,Increase,Above Fair Price,OverValued,Not Sustainable,76.985,0.122815,4.327843e+10,Large,0.81,4.790,0.0660,14.804809,16.072025,19.449588,36.42,0.117663
4,ADP,"Automatic Data Processing, Inc.",Industrials,Increase,Above Fair Price,OverValued,Sustainable,201.010,0.182708,8.443324e+10,Large,0.83,6.460,0.1371,26.483530,31.116098,90.801919,14.18,0.120094


In [10]:
close_panel.head()

,AAP,ABT,ACN,ADM,ADP,ADSK,AEE,AEP,AFL,AJG,...,WRLD,WTFC,WTRH,WW,WWD,XOMA,XRX,ZD,ZUMZ,ESNT
Date,,,,,,,,,,,,,,,,,,,,,
2017-02-03,162.750000,42.779999,114.489998,43.990002,96.870003,84.410004,52.689999,64.029999,34.305000,54.009998,...,49.310001,72.650002,9.901,12.29,70.330002,4.45,28.600000,73.565216,21.150000,35.910000
2017-02-06,160.270004,42.430000,114.180000,44.020000,96.470001,82.820000,52.650002,63.820000,34.465000,54.020000,...,49.139999,71.860001,9.900,12.35,70.010002,4.30,28.280001,73.400002,20.450001,35.720001
2017-02-07,159.179993,42.889999,115.919998,44.590000,96.290001,84.500000,52.680000,63.930000,34.485001,53.930000,...,49.840000,71.599998,9.900,12.31,69.730003,4.21,28.280001,73.886955,20.100000,36.040001
2017-02-08,162.729996,42.400002,115.430000,43.459999,96.580002,82.930000,53.119999,63.799999,34.820000,54.060001,...,50.430000,71.160004,9.900,12.28,69.400002,4.15,29.440001,73.034782,19.950001,36.290001
2017-02-09,164.830002,42.540001,116.980003,43.540001,97.650002,83.879997,52.900002,63.419998,34.950001,54.709999,...,51.259998,72.419998,10.000,12.35,70.389999,4.10,29.040001,74.417389,20.650000,36.240002


In [11]:
AdjClose_panel.head()

,AAP,ABT,ACN,ADM,ADP,ADSK,AEE,AEP,AFL,AJG,...,WRLD,WTFC,WTRH,WW,WWD,XOMA,XRX,ZD,ZUMZ,ESNT
Date,,,,,,,,,,,,,,,,,,,,,
2017-02-03,157.890411,39.270012,105.562996,37.597412,87.912636,84.410004,45.868515,53.538227,30.290247,48.557995,...,49.310001,68.225578,9.901,12.29,68.323280,4.45,23.375423,73.565216,21.150000,34.572044
2017-02-06,155.484421,38.948734,105.277184,37.623055,87.549629,82.820000,45.833706,53.362640,30.431515,48.566975,...,49.139999,67.483673,9.900,12.35,68.012421,4.30,23.113874,73.400002,20.450001,34.389118
2017-02-07,154.426956,39.370998,106.881500,38.110218,87.386269,84.500000,45.859814,53.454613,30.449175,48.486065,...,49.840000,67.370773,9.900,12.31,67.740410,4.21,23.113874,73.886955,20.100000,34.697193
2017-02-08,157.870956,38.921196,106.429733,37.144428,87.649460,82.930000,46.242840,53.842827,30.744982,48.602943,...,50.430000,66.956764,9.900,12.28,67.419815,4.15,24.061970,73.034782,19.950001,34.937889
2017-02-09,159.908234,39.049706,107.858849,37.212811,88.620506,83.879997,46.051334,53.522125,30.859760,49.187328,...,51.259998,68.142334,10.000,12.35,68.381561,4.10,23.735037,74.417389,20.650000,34.889748


# --------- Volatility Calculation ---------

In [ ]:
annualized_historical_volatility()